In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

#Need py38 environment for this to run

import sys
import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

hards_no_cbd = [
        holo.evolution.Hard_GW,
        holo.evolution.Sesana_Scattering(),
        holo.evolution.Dynamical_Friction_NFW(attenuate=False),
        ]

hards_with_cbd = [
        holo.evolution.Hard_GW,
        holo.evolution.Sesana_Scattering(),
        holo.evolution.Dynamical_Friction_NFW(attenuate=False),
        holo.evolution.CBD_Torques()
        ]


pop = holo.population.Pop_Illustris()
N = pop._size
print("N = ", N)

evol_dict_no_cbd = {}
evol_dict_with_cbd = {}

all_eb = [0.0, 0.01, 0.25, 0.50, 0.75, 0.95, 0.99]
for e_b in all_eb:
    eccen = np.ones(N)*e_b
    # ---- Create initial population
    pop = holo.population.Pop_Illustris(eccen=eccen)

    # ---- Apply population modifiers
    redz = cosmo.a_to_z(pop.scafa)
    
    evo = holo.evolution.Evolution(pop, hards_no_cbd, nsteps = 100, debug=True, acc=None)
    evo.evolve()
    evol_dict_no_cbd['eb=%.2f' %e_b] = evo
    
    evo = holo.evolution.Evolution(pop, hards_with_cbd, nsteps = 100, debug=True, acc=None)
    evo.evolve()
    evol_dict_with_cbd['eb=%.2f' %e_b] = evo
    
 

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(15,20), sharex=True)
fs = 30

fname = "eccentricity_evolution_vs_separation"

import seaborn as sns
palette = sns.color_palette('bright')
all_colors = {}

all_eb = [0.0, 0.01, 0.25, 0.50, 0.75, 0.95, 0.99]
color_idx = ['blueviolet', 'steelblue', 'lightseagreen', 'green', 'lime', 'gold', 'firebrick']
for e_b,color in zip(all_eb, color_idx):
    all_colors['eb=%.2f' %e_b] = color

labels = ['No CBD', 'With CBD']
n_bins = 100
sepa_bins = np.logspace(-4, 5.5, n_bins)
for e_b in [0.0, 0.01, 0.25, 0.50, 0.75, 0.95, 0.99]:
    fname += '_%.2f' %e_b
    for ax,evol_dict,label in zip(axes, [evol_dict_no_cbd, evol_dict_with_cbd],labels):
        evo = evol_dict['eb=%.2f' %e_b]
        n_binaries = np.shape(evo.sepa)[0]
        all_eccen_binned = np.zeros((n_binaries, n_bins))
        for n_binary in np.arange(0,n_binaries):
            digitized = np.digitize(evo.sepa[n_binary,:]/PC, sepa_bins)
            eccen_binned = evo.eccen[n_binary,:][digitized]
            all_eccen_binned[n_binary] = np.array(eccen_binned)

        """ FIRST PLOT ECCEN AGAINST SEPARATION """
        perc_50 = np.percentile(all_eccen_binned, 50, axis=0)
        perc_32 = np.percentile(all_eccen_binned, 32, axis=0)
        perc_68 = np.percentile(all_eccen_binned, 68, axis=0)
        ax.semilogx(sepa_bins, perc_50, label = r'$e_{\rm b} = $' + '%.2f' %e_b, color=all_colors['eb=%.2f' %e_b])
        ax.fill_between(sepa_bins, perc_32, perc_68, color=all_colors['eb=%.2f' %e_b], alpha=0.5)
        #ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=int(len(all_eb)/2), fontsize=0.8*fs)
        #ax.legend(fontsize=fs)
        ax.set_xlim([1.e-4, 1.e3])
        ax.set_ylabel('Eccentricity', fontsize=fs)
        plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
        plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
        ticksize = 15
        tickwidth = 2
        ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
        ax.tick_params(axis='both', which='minor', direction='inout', size=0.5*ticksize, width=0.25*tickwidth)
        ax.set_title(label, fontsize=fs)
        #plt.tight_layout()

ax.set_xlabel('Separation [PC]', fontsize=fs)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=int(len(all_eb)/2), fontsize=fs)
plt.tight_layout()
plt.savefig(fname + '.png',  facecolor='w')    
        

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(15,20), sharex=True)
fs = 30

fname = "eccentricity_evolution_vs_frequency"

import seaborn as sns
palette = sns.color_palette('bright')
all_colors = {}

all_eb = [0.0, 0.01, 0.25, 0.50, 0.75, 0.95, 0.99]
color_idx = ['blueviolet', 'steelblue', 'lightseagreen', 'green', 'lime', 'gold', 'firebrick']
for e_b,color in zip(all_eb, color_idx):
    all_colors['eb=%.2f' %e_b] = color

labels = ['No CBD', 'With CBD']
n_bins = 100
freq_bins = np.logspace(-25, -2, n_bins)
for e_b in [0.0, 0.01, 0.25, 0.50, 0.75, 0.95, 0.99]:
    fname += '_%.2f' %e_b
    for ax,evol_dict,label in zip(axes, [evol_dict_no_cbd, evol_dict_with_cbd],labels):
        evo = evol_dict['eb=%.2f' %e_b]
        n_binaries = np.shape(evo.sepa)[0]
        all_eccen_binned = np.zeros((n_binaries, n_bins))
        for n_binary in np.arange(0,n_binaries):
            #print("evo.freq_orb_rest[n_binary,:]", evo.freq_orb_rest[n_binary,:])
            #print("freq_bins", freq_bins)
            digitized = np.digitize(evo.freq_orb_rest[n_binary,:], freq_bins)
            #print("digitized", digitized)
            eccen_binned = evo.eccen[n_binary,:][digitized]
            all_eccen_binned[n_binary] = np.array(eccen_binned)

        """ FIRST PLOT ECCEN AGAINST FREQUENCY """
        perc_50 = np.percentile(all_eccen_binned, 50, axis=0)
        perc_32 = np.percentile(all_eccen_binned, 32, axis=0)
        perc_68 = np.percentile(all_eccen_binned, 68, axis=0)
        ax.loglog(freq_bins, perc_50, label = r'$e_{\rm b} = $' + '%.2f' %e_b, color=all_colors['eb=%.2f' %e_b])
        ax.fill_between(freq_bins, perc_32, perc_68, color=all_colors['eb=%.2f' %e_b], alpha=0.5)
        #ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=int(len(all_eb)/2), fontsize=0.8*fs)
        #ax.legend(fontsize=fs)
        ax.set_xlim([1.e-13, 1.e-2])
        ax.set_ylim([1.e-4, 1.0])
        ax.set_ylabel('Eccentricity', fontsize=fs)
        plt.setp(ax.get_xticklabels(which='both'), fontsize=fs, rotation=45)
        plt.setp(ax.get_yticklabels(which='both'), fontsize=fs)
        ticksize = 15
        tickwidth = 2
        ax.tick_params(axis='both', which='major', direction='inout', size=ticksize, width=tickwidth)
        ax.tick_params(axis='both', which='minor', direction='inout', size=0.5*ticksize, width=0.25*tickwidth)
        ax.set_title(label, fontsize=fs)
        #plt.tight_layout()

ax.set_xlabel('Frequency [Hz]', fontsize=fs)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=int(len(all_eb)/2), fontsize=fs)
plt.tight_layout()
plt.savefig(fname + '.png',  facecolor='w')    
        